In [ ]:
!pip install datasets --upgrade --quiet
!pip install googletrans==4.0.0-rc1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import json
from googletrans import Translator

In [ ]:
ds = load_dataset("OpenAssistant/oasst2")
train = ds['train'].to_pandas()
val = ds['validation'].to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/128575 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6599 [00:00<?, ? examples/s]

In [ ]:
en_train = train[train.lang == 'en']
en_val = val[val.lang == 'en']

In [ ]:
# returns list of format ["user", "assistant", "user", "assistant label"]
# the last index is the label and the rest is the input
# the first index always starts with user prompt
def prep_data(dataset:pd.DataFrame):
  result = []
  copy = dataset.set_index("message_id")
  asst_reply = dataset[dataset.role == "assistant"]

  for _, row in asst_reply.iterrows():
    try:
      conversation = [row.text]
      parent_id = row.parent_id
      while parent_id:
        parent_row = copy.loc[parent_id]
        conversation.insert(0, parent_row.text)
        parent_id = parent_row.parent_id

      result.append(conversation)
    except:
      pass

  return result


In [ ]:
result = prep_data(en_val)

In [ ]:
save_path = "/content/drive/MyDrive/Datasets/OASST2/Processed/"

file_name = "EN-train.json"


In [ ]:

with open(save_path + file_name, 'w') as f:
  json.dump(result, f)


In [ ]:
with open(save_path + file_name, 'r') as f:
  result = json.load(f)
  print(result[0])


['I am making mayonnaise, it was starting to thicken but now it has become runny and liquid again, is there any way to salvage it?', "Yes, it's possible to fix runny mayonnaise! The most common reason for mayonnaise becoming runny is because the oil was added too quickly or the egg yolk wasn't emulsified properly. Here are some steps you can take to fix it:\n\n1. Separate another egg yolk and place it in a clean, dry bowl.\n2. Slowly add the runny mayonnaise to the egg yolk while whisking vigorously.\n3. Once all the runny mayonnaise has been added, continue whisking until the mixture has emulsified and thickened.\n4. If the mayonnaise is still too runny, you can add another egg yolk and repeat the process.\n\nIf the mayonnaise still won't thicken, you can try adding a small amount of dijon mustard or vinegar to the mixture, which can act as emulsifiers and help stabilize the mayonnaise. It's important to add these ingredients slowly and in small amounts to avoid over-thinning the mixt

In [ ]:
translator = Translator()

def translate(text):
  return translator.translate(text, src='en', dest='id').text

In [ ]:
id_train = en_train.copy()
id_val = en_val.copy()

In [ ]:
val_temp_filename = "ID_gt-val-temp.json"
train_temp_filename = "ID_gt-train-temp.json"


with open(save_path + val_temp_filename, 'r') as f:
  id_val_text = json.load(f)

try:
  with open(save_path + train_temp_filename, 'r') as f:
    id_train_text = json.load(f)
except:
  id_train_text=[]

In [ ]:

batch = 100
for i in range(len(id_val_text), len(id_val)):
  if i % batch == 0:
    print(i)
    with open(save_path + val_temp_filename, 'w') as f:
      json.dump(id_val_text, f)

  err=20
  while err>0:
    try:
      id_val_text.append(translate(id_val.iloc[i].text))
      err=-1
    except:
      if err > 18:
        print(f"[{i}]", end="")
      err -= 1

  if err == 0:
    id_val_text.append(id_val.iloc[i].text)
    print(f"/{i}/", end="")
  print("=", end="")
with open(save_path + val_temp_filename, 'w') as f:
  json.dump(id_val_text, f)

200
====================================================================================================300
====================================================================================================400
===================================[435][435]/435/===[438][438]/438/==============================================================500
====================================================================================================600
====================================================================================================700
====================================================================================================800
====================================================================================================900
====================================================================================================1000
====================================================================================================1100
============================

In [ ]:
errlist=[]
batch = 100
for i in range(len(id_train_text), len(id_train)):
  if i % batch == 0:
    print(i)
    with open(save_path + train_temp_filename, 'w') as f:
      json.dump(id_train_text, f)

  err=20
  while err>0:
    try:
      id_train_text.append(translate(id_train.iloc[i].text))
      err=-1
    except:
      if err > 18:
        print(f"[{i}]", end="")
      err -= 1

  if err == 0:
    id_train_text.append(id_train.iloc[i].text)
    errlist.append(i)
    print(f"/{i}/", end="")
  print("=", end="")
with open(save_path + train_temp_filename, 'w') as f:
  json.dump(id_train_text, f)

51700
====================================================================================================51800
====================================================================================================51900
====================================================================================================52000
========================================[52040][52040]/52040/============================================================52100
====================================================================================================52200
====================================================================================================52300
====================================================================================================52400
====================================================================================================52500
=======================[52523][52523]/52523/=============================================================================5260

In [ ]:
id_train.text = id_train_text

In [ ]:
id_val.text = id_val_text

In [ ]:
id_train_df_filename = "ID-train-EN_GT.df.csv"
id_train.to_csv(save_path + id_train_df_filename)

In [ ]:
id_val_df_filename = "ID-val-EN_GT.df.csv"
id_val.to_csv(save_path + id_val_df_filename)

In [ ]:
id_train_prepped = prep_data(id_train)
id_val_prepped = prep_data(id_val)

In [ ]:
id_train_filename = "ID-train-EN_GT.json"
id_val_filename = "ID-val-EN_GT.json"

with open(save_path + id_train_filename, 'w') as f:
  json.dump(id_train_prepped, f)
with open(save_path + id_val_filename, 'w') as f:
  json.dump(id_val_prepped, f)
